# Toronto Neighborhoods
## Import libraries

In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import folium
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Libraries imported!")

Libraries imported!


## Setting up Data
The data is scraped from [this Wikipedia Page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). Postal codes with no boroughs assigned are dropped and the column names and contents are modified into the desired format.

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(res)
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

df.rename(columns={'Postal code':'PostalCode'},inplace=True)

df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)

df.Neighborhood = df.Neighborhood.str.replace(" /",",")

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df.shape

(103, 3)

## Importing Latitude and Longitudes
Using the geocoder package was unsuccessful, so we will just download the CSV file and merge it with our original dataframe.

In [4]:
lat_lng = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lng.rename(columns={'Postal Code':'PostalCode'},inplace=True)
lat_lng.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df = df.merge(lat_lng)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [6]:
df.shape

(103, 5)

## Neighborhood Analysis
### Restrict Data
Let's restrict our data to only those that contain Toronto in the name of the Borough

In [7]:
df = df[df.Borough.str.contains("Toronto")]
df = df.reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [8]:
df.shape

(39, 5)

### Visualizing the Map
Let's create a folium map object with markers for each postal code that have popups with the names of the neighborhoods within them.

*Note: Folium Maps will not render properly in GitHub. Follow the link [here](https://nbviewer.jupyter.org/github/alexanderWhile/Coursera_Capstone/blob/master/toronto_neighborhoods.ipynb) to see the full output*

In [9]:
m = folium.Map(
    location=[43.6532, -79.3832],
    zoom_start=11)

for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['Latitude'],df.iloc[i]['Longitude']],
        popup=df.iloc[i]['Neighborhood']
        ).add_to(m)

m

### Connect to Foursquare
Define credentials and version

In [10]:
CLIENT_ID = 'IA4SDU5HX0UHCL4VSZJDAHBXWJHJY4HPTFNBLWHG4YHYSLWH'
CLIENT_SECRET = '21PID34DCUTLYIWC2RRRRWMBKIE1ZUUXQKE2ZEAASQ4VIWX5'
VERSION = '20200402'

print("Client ID:",CLIENT_ID)
print("Client Secret:", CLIENT_SECRET)

print("Version:", VERSION)

Client ID: IA4SDU5HX0UHCL4VSZJDAHBXWJHJY4HPTFNBLWHG4YHYSLWH
Client Secret: 21PID34DCUTLYIWC2RRRRWMBKIE1ZUUXQKE2ZEAASQ4VIWX5
Version: 20200402


Let's look at the top venues near our first postal code's neighborhoods.

In [11]:
neighborhood = df.loc[0,'Neighborhood']
neighborhood_lat = df.loc[0,'Latitude']
neighborhood_lng = df.loc[0,'Longitude']

LIMIT = 100
radius = 500

print("Neighborhood(s):", neighborhood)
print("Latitude:", neighborhood_lat)
print("Longitude:", neighborhood_lng)

Neighborhood(s): Regent Park, Harbourfront
Latitude: 43.6542599
Longitude: -79.3606359


Define url and make request

In [12]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_lat,
    neighborhood_lng,
    radius,
    LIMIT)

results = requests.get(url).json()
print("Success!")

Success!


Define function for parsing venue category

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

print("Function Defined!")

Function Defined!


Gather venues and format into a clean data frame

In [14]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


Check the number of venues returned.

In [38]:
print("There are", nearby_venues.shape[0], "venues nearby")

There are 49 venues nearby


Let's make a map of this postal code and the nearby venues. 

In [16]:
neighborhood_map = folium.Map(
    location=[neighborhood_lat,neighborhood_lng],
    zoom_start=15)

folium.vector_layers.CircleMarker(
    [neighborhood_lat,neighborhood_lng],
    radius=10,
    color='red',
    popup=neighborhood,
    fill = True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(neighborhood_map)

for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.vector_layers.CircleMarker(
        [lat,lng],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(neighborhood_map)

neighborhood_map

Now we will define a function that returns the nearby venues for all of our postal codes. We will print the name of each neighborhood to ensure successful iteration. 

In [17]:
def get_nearby_venues(names, latitudes, longitudes):

    venues_list = []
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        results = requests.get(url).json()['response']['groups'][0]['items']

        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])

    nearby_venues.columns = ['Neighborhood',
                    'Neighborhood Latitude',
                    'Neighborhood Longitude',
                    'Venue',
                    'Venue Latitude',
                    'Venue Longitude',
                    'Venue Category']
   
    return(nearby_venues)

print("Success!")



Success!


Time to implement the function and save the results in a new data frame.

In [18]:
toronto_venues = get_nearby_venues( names=df.Neighborhood,
                                    latitudes = df.Latitude,
                                    longitudes = df.Longitude)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

Let's check the shape of the new data frame and preview the results.

In [19]:
print(toronto_venues.shape)
toronto_venues

(1622, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
...,...,...,...,...,...,...,...
1617,Business reply mail Processing CentrE,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
1618,Business reply mail Processing CentrE,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1619,Business reply mail Processing CentrE,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
1620,Business reply mail Processing CentrE,43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher


Now we will count how many venues were returned for each neighborhood.

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,64,64,64,64,64,64
Christie,18,18,18,18,18,18
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


How many unique venue categories were there?

In [21]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 227 unique categories


Now let us use one-hot encoding to determine how which venue categories are present in each neighborhood. 

In [22]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

cols = list(toronto_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))

toronto_onehot = toronto_onehot.loc[:,cols]
toronto_onehot.head()


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_onehot.shape

(1622, 227)

Now we will determine the frequency of each venue category for each neighborhood.

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000,0.015625
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026667
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Let's look at the top 5 venues for each neighborhood. 

In [25]:
num_top_venues = 5

for hood in toronto_grouped.Neighborhood:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped.Neighborhood == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.05
1  Farmers Market  0.04
2    Cocktail Bar  0.04
3      Restaurant  0.04
4        Beer Bar  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3          Bakery  0.04
4       Pet Store  0.04


----Business reply mail Processing CentrE----
            venue  freq
0      Comic Shop  0.06
1   Garden Center  0.06
2            Park  0.06
3      Restaurant  0.06
4  Farmers Market  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2    Airport Terminal  0.14
3             Airport  0.07
4  Airport Food Court  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.19
1                Café  0.06
2  Italian Restaurant  0.06
3      Sandwich Pl

For simplicity, let's define a function to return the top *n* venues found in a neighborhood.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

print("Function Defined!")

Function Defined!


Let's test out the function on the first neighborhood.

In [27]:
 return_most_common_venues(toronto_grouped.loc[0],3)

array(['Coffee Shop', 'Farmers Market', 'Cheese Shop'], dtype=object)

We will now create a data frame containing the top 10 most common venues for each neighborhood.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in range(0,num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted.Neighborhood = toronto_grouped.Neighborhood

for ind in range(0,toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.loc[ind], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Grocery Store,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,Business reply mail Processing CentrE,Skate Park,Pizza Place,Butcher,Burrito Place,Fast Food Restaurant,Farmers Market,Auto Workshop,Spa,Restaurant,Garden
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina,Boat or Ferry,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place


Now we use *k*-means clustering to group the neighborhoods into 5 clusters based on their most common venues. 

In [29]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Merge the clustered data frame with our original.

In [30]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df 

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')

toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Bar,Sushi Restaurant,Pizza Place
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,2,Pharmacy,Bakery,Grocery Store,Recording Studio,Café,Supermarket,Middle Eastern Restaurant,Bar,Bank,Brewery


Now let's map our resulting clusters. 

In [31]:
map_clusters = folium.Map(
    location=[43.6532, -79.3832],
    zoom_start= 11
    )

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged.Latitude, toronto_merged.Longitude, toronto_merged.Neighborhood, toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7
).add_to(map_clusters)


map_clusters

### Analyzing Clusters.
Here we will look at each cluster and describe it based on its venues.

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,0,Park,Swim School,Bus Line,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
21,Forest Hill North & West,0,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


We will call this the "Park" cluster as both neighborhoods most common venue is parks.

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,1,Home Service,Garden,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


This cluster only contains one neighborhood, Roselawn, so we will just call this "Roselawn". It seems to have a fair amount of ethinc food available in the area. 

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",2,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Theater,Shoe Store
1,"Queen's Park, Ontario Provincial Government",2,Coffee Shop,Diner,Fried Chicken Joint,Burrito Place,Juice Bar,Sandwich Place,Café,Italian Restaurant,Beer Bar,Mexican Restaurant
2,"Garden District, Ryerson",2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore,Restaurant
3,St. James Town,2,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Park
4,The Beaches,2,Health Food Store,Trail,Pub,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
5,Berczy Park,2,Coffee Shop,Farmers Market,Cheese Shop,Beer Bar,Bakery,Café,Restaurant,Italian Restaurant,Seafood Restaurant,Cocktail Bar
6,Central Bay Street,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Spa,Ice Cream Shop,Japanese Restaurant,Sushi Restaurant,Bubble Tea Shop,Salad Place
7,Christie,2,Grocery Store,Café,Park,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Athletics & Sports,Restaurant
8,"Richmond, Adelaide, King",2,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Deli / Bodega,Bar,Sushi Restaurant,Pizza Place
9,"Dufferin, Dovercourt Village",2,Pharmacy,Bakery,Grocery Store,Recording Studio,Café,Supermarket,Middle Eastern Restaurant,Bar,Bank,Brewery


This cluster shows a lot of Coffee Shops and Cafes in their most common venues, as well as many resaurants. We will call this the "Food" cluster. 

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Rosedale,3,Park,Playground,Trail,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


This also contains one neighborhood, so we will call it "Rosedale". It contains many fitness activities, though it is curious why this was not included in the first cluster. 

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",4,Playground,Trail,Tennis Court,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Similarly we will call this the "Moore Park and Summerhill East" cluster. Like the Rosedale cluster, it could have likely been combined with others if we use less clusters. 